In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

features = pd.read_csv('features.csv', index_col='match_id')
print("The first 5 rows of dataset")
features.head(5)

The first 5 rows of dataset


,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_hero,d1_level,d1_xp,d1_gold,d1_lh,d1_kills,d1_deaths,d1_items,d2_hero,d2_level,d2_xp,d2_gold,d2_lh,d2_kills,d2_deaths,d2_items,d3_hero,d3_level,d3_xp,d3_gold,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,1,11,4,3,1058,996,12,0,0,6,42,4,1085,986,12,0,0,4,21,5,2052,1536,23,0,0,6,37,3,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7,1,9,NaN,134,-80,244,2,2,2,0,35,103,-84,221,3,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,0,5,39,5,1960,1384,16,0,0,8,88,3,640,566,1,0,1,5,79,3,720,1350,2,2,0,12,7,2,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54,1,7,NaN,173,-80,NaN,2,0,2,0,-20,149,-84,195,5,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,0,9,22,5,2305,2028,19,1,1,10,66,3,1024,959,19,0,1,10,86,3,755,620,3,0,0,8,29,4,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224,0,3,NaN,63,-82,NaN,2,5,2,1,-39,45,-77,221,3,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,0,6,96,5,1878,1174,17,0,0,6,48,3,732,1468,22,0,0,10,15,4,1681,1051,11,0,0,7,102,2,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208,-75,NaN,0,3,2,0,-30,124,-80,184,0,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,0,8,26,3,704,586,1,0,2,9,69,3,1169,1665,20,1,0,7,22,3,1055,638,1,0,0,9,25,5,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21,1,6,NaN,166,-81,181,1,4,2,0,46,182,-80,225,6,3,3,0,-16,2449,0,4,1974,3,63


In [2]:
total_rows = len(features.index)
df = features.count()
print("Features with empty values")
df.loc[df < 97230]

Features with empty values


first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

In [3]:
df = features.fillna(0)
df.drop(['radiant_win', 'duration', 'tower_status_radiant', 'tower_status_dire',
         'barracks_status_radiant', 'barracks_status_dire'], inplace=True, axis=1)
X = df
Y = features.radiant_win

## Gradient Boosting

In [4]:
start_time = datetime.datetime.now()
kf = KFold(len(df), n_folds=5, shuffle=True)
clf = GradientBoostingClassifier(n_estimators=30)
clf.fit(X, Y)
score_mean = cross_val_score(estimator=clf, X=X, y=Y, cv=kf, scoring='roc_auc').mean()
print(score_mean)
print(datetime.datetime.now() - start_time)

0.6888015049
0:04:23.114000


## Logistic Regression

In [5]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
kf = KFold(len(df), n_folds=5, shuffle=True)
c = 0.01
clf = LogisticRegression(penalty='l2', C=c)
clf.fit(X, Y)
score_mean = cross_val_score(estimator=clf, X=X, y=Y, cv=kf, scoring='roc_auc').mean()
print("c = " + str(c) + " score = " + str(score_mean))

c = 0.01 score = 0.716416738913


### Removed categorical features

In [6]:
X = df.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
             'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
scaler.fit(X)
X = scaler.transform(X)
c = 0.01
clf.fit(X, Y)
score_mean = cross_val_score(estimator=clf, X=X, y=Y, cv=kf, scoring='roc_auc').mean()
print("c = " + str(c) + " score = " + str(score_mean))

c = 0.01 score = 0.716477473194


### Unique heroes id

In [7]:
hero_fields_list = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                    'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
set_id = set()
for field in hero_fields_list:
    heroes_list = df[field].unique().tolist()
    for hid in heroes_list:
        set_id.add(hid)
num_unique_id = len(set_id)
print("num_unique_id = " + str(num_unique_id))

num_unique_id = 108


### Bag of Words

In [8]:
N = 113  # heroes count in Dota 2
X_pick = np.zeros((len(df), N))
for i, match_id in enumerate(df.index):
    for p in range(5):
        X_pick[i, df.ix[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
        X_pick[i, df.ix[match_id, 'd%d_hero' % (p + 1)] - 1] = -1
X_new = np.concatenate([X, X_pick], axis=1)
kf = KFold(len(df), n_folds=5, shuffle=True)
c = 0.01
clf = LogisticRegression(penalty='l2', C=c)
clf.fit(X_new, Y)
score_mean = cross_val_score(estimator=clf, X=X_new, y=Y, cv=kf, scoring='roc_auc').mean()
print("c = " + str(c) + " score = " + str(score_mean))

c = 0.01 score = 0.751554711435


### Test data

In [10]:
data = pd.read_csv("features_test.csv", index_col='match_id')
df = data.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',
                'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
X_test = df.fillna(0)
scaler.fit(X_test)
X_test = scaler.transform(X_test)
X_pick = np.zeros((len(data), N))
for i, match_id in enumerate(data.index):
    for p in range(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p + 1)] - 1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p + 1)] - 1] = -1
X_new = np.concatenate([X_test, X_pick], axis=1)

In [12]:
answer = clf.predict_proba(X_new)[:, 1]
id_match = data.index
print('match_id,radiant_win')
min_proba = 1
max_proba = 0
for i in range(0, len(data)):
    current = answer[i]
    print(str(id_match[i]) + "," + str(current))
    if current > max_proba:
        max_proba = current
    if current < min_proba:
        min_proba = current
print('Min: ' + str(min_proba))
print('Max: ' + str(max_proba))

match_id,radiant_win
6,0.81396275121
7,0.74535289436
10,0.195020352311
13,0.870518821015
16,0.241226437985
18,0.381528816111
19,0.522050062063
24,0.566261802369
33,0.222147837812
37,0.651234759492
41,0.166839746551
42,0.332432468119
55,0.24355568133
60,0.67756920628
62,0.552660624326
63,0.533538870467
64,0.102526337791
71,0.549210052041
72,0.327608100037
83,0.505883244249
85,0.778716043234
89,0.923512843655
92,0.743805115403
100,0.953558402607
102,0.870967164629
108,0.540153288641
111,0.78370274747
126,0.170444286781
130,0.0499126366127
140,0.786677579914
141,0.262776314374
152,0.837216567013
153,0.727563208073
159,0.55883841885
160,0.559962262824
163,0.589613723572
178,0.552399765729
179,0.378026004311
181,0.688021490361
196,0.155470000925
206,0.11986575452
209,0.544503281871
216,0.569823204805
228,0.0787127203076
230,0.419978587607
242,0.709147893284
245,0.170860917459
261,0.4993405585
274,0.470338009039
282,0.733387818797
289,0.51335223834
296,0.101882406141
297,0.600767327806
302,0